# simulation with ivs data

### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from shapely.geometry import Point
import shapely
import geopandas as gpd
import movingpandas as mpd

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pickle

# OpenTNSIM
import opentnsim
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot


# dtv_backend
import dtv_backend.fis as fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate

# Used for mathematical functions
import math             
import numpy as np

# Used for making the graph to visualize our problem
import networkx as nx  

### Create graph

In [ ]:
#Load lock info
# url = "https://zenodo.org/records/6673604/files/FIS_locks_grouped.geojson?download=1"
# import requests
# import io
# resp = requests.get(url)
# stream = io.BytesIO(resp.content)
# locks_gdf = gpd.read_file(stream)

In [3]:
#Load graph
graph1 =pickle.load(open("data\euris\euris_graph_v0.1.pickle", "rb"))

# remove edge ('NL5245', 'NLJ1934')
graph1.remove_edge((("NL"),('J5245')), (("NL"),('J1934')))

graph = graph_module.Graph()
graph.graph = graph1
graph.graph_info = opentnsim.utils.info(graph.graph)

In [24]:
print(graph.graph[('RO', 'D5940')])

{('RO', 'D7500'): {'name': 'Danube 1075-1059.4 (RS-RO)', 'name_cb': None, 'cntrycode': 'RO', 'cntrycode_cb': 'RS', 'fw_code': '00001', 'fw_code_cb': '00001', 'seq_nr': 'D7500', 'seq_nr_cb': '10750', 'code_cb': 'RS0000110750', 'ww_name': 'Dunărea', 'ww_name_cb': 'Dunav', 'rt_name': 'Route Danube', 'rt_name_cb': 'Route Danube', 'wwauthorit': 'RNA Drobeta', 'wwauthorit_cb': 'Direkcija za vodne puteve - Plovput', 'cemt': 'VII', 'mdraughtcm': 250.0, 'mlengthcm': 14000.0, 'mlencon': 30000.0, 'mwidthcm': 1200.0, 'mwidcon': 3300.0, 'speed': '18', 'speedcon': '18', 'calspeed_up': 8.0, 'calspeed_down': 15.0, 'calspeedc_up': 8.0, 'calspeedc_down': 15.0, 'maxspeed_up': nan, 'maxspeed_down': nan, 'maxspeedc_up': nan, 'maxspeedc_down': nan, 'tidedep': 0, 'tot_length': nan, 'estuary': nan, 'active': 1, 'ww_charges': nan, 'remark': None, 'istentec': None, 'geometry': <LINESTRING (21.36 44.821, 21.36 44.82, 21.361 44.82, 21.361 44.82, 21.361 4...>, 'dataset': 'FairwaySection', 'date': '20240527', 'leng

In [39]:
# save afzetting in geopanda
afzetting = pd.DataFrame({'A': graph1.nodes[(("NL"),('J5245'))],
'B': graph1.nodes[(("NL"),('J1934'))]}).T
afzetting = gpd.GeoDataFrame(afzetting[['ww_name', 'rt_name', 'locode', 'objectname', 'sectionref', 'hectom', 'geometry', 'date']])
afzetting.to_file('dtv_backend/plots_routes/afzetting_locatie')

In [42]:
# save graph in geopanda
df_edges = nx.to_pandas_edgelist(graph1)
gpd_edges = gpd.GeoDataFrame(df_edges)
gpd_edges['target'] = gpd_edges.target.str.join("_") # shape file cannot handle tuples
gpd_edges['source'] = gpd_edges.source.str.join("_") # shape file cannot handle tuples
gpd_edges.to_file('dtv_backend/plots_routes/edges_met_afzetting')


D:\Users\ALPF\AppData\Local\Temp\ipykernel_16896\2826873833.py:6: UserWarning:

Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.



### Make vessels and paths

##### read data

In [44]:
# lees data in.
data = gpd.read_file("dtv_backend\data\ivs\ivs-2024-geocoded-sample.gpkg")

#filter data op bestaande iso datum en geometry.
data['datetime'] = pd.to_datetime(data['v05_06_begindt_evenement_iso'], format = 'ISO8601', errors = 'coerce')
data.dropna(subset = ['datetime', 'geometry'], inplace = True)


D:\Users\ALPF\AppData\Local\Temp\ipykernel_16896\114855860.py:5: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



filter only on rotterdam to duisburg, 2023. Only keep 5 ships

In [45]:
data = data[
    (data.UNLO_herkomst== "NLDOR")
    #(data.UNLO_herkomst== "NLGAM")
     & (data.UNLO_bestemming.str.startswith("DE"))
     & (data.v05_06_begindt_evenement_iso.str.startswith("2023"))
     ].head()
data

,Jaarmaand,Jaar,Maand,Weeknr,v05_06_begindt_evenement_iso,v05_06_Begindt_evenement,UNLO_herkomst,UNLO_bestemming,v15_1_Scheepstype_RWS,SK_CODE,v18_Laadvermogen,v28_Beladingscode,v38_Vervoerd_gewicht,v30_4_Containers_TEU_S,nstr_nw,nst2007_nw,geometry,datetime
14694,2308,2023,8,32,2023-08-10T08:00:00+02:00,10 augustus 2023 08:00:00 uur,NLDOR,DEDUI,9,C3l,4320.0,5.0,0.0,0.0,9,16,"LINESTRING (4.63333 51.78333, 6.75000 51.46667)",2023-08-10 08:00:00+02:00
20814,2301,2023,1,1,2023-01-04T16:00:00+01:00,04 januari 2023 16:00:00 uur,NLDOR,DEMHG,9,C3l,5730.0,7.0,3550000.0,0.0,2,2.1,"LINESTRING (4.63333 51.78333, 8.45000 49.48333)",2023-01-04 16:00:00+01:00
27391,2309,2023,9,39,2023-09-25T10:00:00+02:00,25 september 2023 10:00:00 uur,NLDOR,DENSS,2,M3,690.0,5.0,0.0,0.0,9,16,"LINESTRING (4.63333 51.78333, 6.68333 51.18333)",2023-09-25 10:00:00+02:00
32727,2309,2023,9,39,2023-09-26T06:00:00+02:00,26 september 2023 06:00:00 uur,NLDOR,DEGEK,1,M2,510.0,7.0,375000.0,0.0,0,1.1,"LINESTRING (4.63333 51.78333, 7.11667 51.51667)",2023-09-26 06:00:00+02:00
35894,2308,2023,8,32,2023-08-12T15:00:00+02:00,12 augustus 2023 15:00:00 uur,NLDOR,DEFRA,21,BII-2L,5320.0,5.0,0.0,0.0,9,16,"LINESTRING (4.63333 51.78333, 8.68333 50.11667)",2023-08-12 15:00:00+02:00


##### Create vessels

In [98]:
path

[('NL', 'J2676'),
 ('NL', 'J2140'),
 ('NL', 'J2139'),
 ('NL', 'J2138'),
 ('NL', 'J2137'),
 ('NL', 'J2136'),
 ('NL', 'J2323'),
 ('NL', 'J2320'),
 ('NL', 'J2319'),
 ('NL', 'J2312'),
 ('NL', 'J2311'),
 ('NL', 'J2310'),
 ('NL', 'J2309'),
 ('NL', 'J2322'),
 ('NL', 'J4495'),
 ('NL', 'J2321'),
 ('NL', 'J2174'),
 ('NL', 'J2402'),
 ('NL', 'J2401'),
 ('NL', 'J2400'),
 ('NL', 'J2399'),
 ('NL', 'J2398'),
 ('NL', 'J3544'),
 ('NL', 'J2408'),
 ('NL', 'J0871'),
 ('NL', 'J3308'),
 ('NL', 'J2016'),
 ('NL', 'J2407'),
 ('NL', 'J2406'),
 ('NL', 'J4235'),
 ('NL', 'J4236'),
 ('NL', 'J2397'),
 ('NL', 'J4230'),
 ('NL', 'J2394'),
 ('NL', 'J2162'),
 ('NL', 'J4227'),
 ('NL', 'J4228'),
 ('NL', 'J1609'),
 ('NL', 'J1236'),
 ('NL', 'J2158'),
 ('NL', 'J1813'),
 ('NL', 'J1684'),
 ('NL', 'J1835'),
 ('NL', 'J1839'),
 ('NL', 'J1834'),
 ('NL', 'J3499'),
 ('NL', 'J1822'),
 ('NL', 'J1821'),
 ('NL', 'J1823'),
 ('NL', 'J4508'),
 ('NL', 'J4506'),
 ('NL', 'J1819'),
 ('NL', 'J1817'),
 ('NL', 'J1816'),
 ('NL', 'J1818'),
 ('NL', 'J

In [100]:
graph.graph.edges[(("NL"),("J1609")), (("NL"), ("J1236"))]

{'name': 'Merwedekanaal (bezuiden de Lek)',
 'name_cb': None,
 'cntrycode': 'NL',
 'cntrycode_cb': None,
 'fw_code': '00104',
 'fw_code_cb': None,
 'seq_nr': '01804',
 'seq_nr_cb': None,
 'code_cb': None,
 'ww_name': 'Merwedekanaal (bezuiden de Lek)',
 'ww_name_cb': None,
 'rt_name': 'Merwedekanaal (bezuiden de Lek)',
 'rt_name_cb': None,
 'wwauthorit': 'RIS-NL',
 'wwauthorit_cb': None,
 'cemt': 'IV',
 'mdraughtcm': 280.0,
 'mlengthcm': 10000.0,
 'mlencon': 10000.0,
 'mwidthcm': 950.0,
 'mwidcon': 950.0,
 'speed': 'Beroepsvaart  : 12 km/u.\nRecreatievaart: 12 km/u.',
 'speedcon': None,
 'calspeed_up': 10.0,
 'calspeed_down': 10.0,
 'calspeedc_up': 10.0,
 'calspeedc_down': 10.0,
 'maxspeed_up': 12.0,
 'maxspeed_down': 12.0,
 'maxspeedc_up': nan,
 'maxspeedc_down': nan,
 'tidedep': 0,
 'tot_length': 4556.0,
 'estuary': 0.0,
 'active': 1,
 'ww_charges': 0.0,
 'remark': None,
 'istentec': None,
 'geometry': <LINESTRING (4.997 51.871, 4.997 51.87, 4.997 51.869, 4.996 51.867, 4.995 51...>,
 

In [108]:
# Make a class out of mix-ins
TransportResource = type('TransportResource', 
                         (core.Identifiable, core.ContainerDependentMovable, 
                          core.HasResource, core.Routable,
                          core.VesselProperties,
                         core.ExtraMetadata), 
                         {})

#define speed: 
def compute_v_provider(v_empty, v_full):
    return lambda x: 1

# define weight
def compute_weight(origin, target, dictionary):
    if dictionary['cemt'] == 'IV': #niet langs te kleine wegen
        return None
    elif origin[1] == 'J0103':
        return None
    else:
        return dictionary['length_m']

vessels = []
for index, row in data.iterrows():
    #determine path
    point_1 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[0]))
    point_2 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[-1]))
    path = nx.dijkstra_path(graph.graph, point_1[0], point_2[0], weight = compute_weight) 
    #determine capacity
    capacity = max(row.v18_Laadvermogen*1000, row.v38_Vervoerd_gewicht, 1)
    data_vessel = {"env": None,
               "name": row.name,
               "type": row['v15_1_Scheepstype_RWS'],
               "B": 1,
               "L": 10,
               "route": path,
               "geometry": Point(row.geometry.coords[0]),  # lon, lat
               "capacity": capacity,
               "v": 1,
               "compute_v": compute_v_provider(v_empty=1, v_full=1),
               }
    vessel = TransportResource(**data_vessel)
    vessels.append(vessel)



# sluis maken


Een sluis maken is mogelijk. Hiervoor moet de sluismodule wel worden aangepast. Dit is nu niet in dit script verwerkt

### Start simulation

In [109]:
def start(env, vessel):
    while True:
        vessel.log_entry_v0("Start sailing", env.now, "", vessel.geometry)
        yield from vessel.move()
        vessel.log_entry_v0("Stop sailing", env.now, "", vessel.geometry)
        
        if vessel.geometry == nx.get_node_attributes(env.FG, "geometry")[vessel.route[-1]]:
            break

In [110]:
# Start simpy environment
simulation_start = datetime.datetime.now()
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.epoch = time.mktime(simulation_start.timetuple())


# Add graph to environment
#graph.add_resources(list(graph.graph.edges), np.ones(len(list(graph.graph.edges))), env)
env.FG = graph.graph



In [111]:

vessels = vessels[0:3]
for i, vessel in enumerate(vessels):
    # Add environment and path to the vessel
    vessel.env = env

    # Add the movements of the vessel to the simulation
    env.process(start(env, vessel))


env.epoch = datetime.datetime.now() #TODO standaard datum kiezen.

env.run()


### Obtain vessel log information
The cel below uses the vessel log. The core function *log_entry* is used, which takes four arguments:

- **Log.** A text to describe what is logged.
- **t.** The timestamp.
- **Value.**  The value for the log (for sailing this is the distance).
- **Geometry** The location of the vessel while loggin.

In [113]:
vessel_log = gpd.GeoDataFrame(vessels[0].logbook, geometry='Geometry')
vessel_log = mpd.Trajectory(vessel_log, traj_id='vessel_1_trip_1', obj_id='vessel_1', t='Timestamp')
#vessel_log.to_line_gdf()
vessel_log.to_line_gdf().to_file('dtv_backend/plots_routes/dordrecht_afzetting_met_weight_function.gpkg')

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-py3.10\lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.



In [ ]:
vessels[0].logbook


[{'Message': 'Start sailing',
  'Timestamp': datetime.datetime(2024, 6, 18, 10, 11, 45),
  'Value': '',
  'Geometry': <POINT (4.633 51.783)>},
 {'Message': 'Sailing to start',
  'Timestamp': datetime.datetime(2024, 6, 18, 10, 18, 0, 340248),
  'Value': 375.34024804315567,
  'Geometry': <POINT (4.635 51.787)>},
 {'Message': "Sailing from node ('NL', 'J2676') to node ('NL', 'J2140') sub edge 0 start",
  'Timestamp': datetime.datetime(2024, 6, 18, 10, 18, 0, 340248),
  'Value': 0,
  'Geometry': <POINT (4.635 51.787)>},
 {'Message': "Sailing from node ('NL', 'J2676') to node ('NL', 'J2140') sub edge 0 stop",
  'Timestamp': datetime.datetime(2024, 6, 18, 10, 22, 51, 189663),
  'Value': 0,
  'Geometry': <POINT (4.637 51.789)>},
 {'Message': "Sailing from node ('NL', 'J2676') to node ('NL', 'J2140') sub edge 1 start",
  'Timestamp': datetime.datetime(2024, 6, 18, 10, 22, 51, 189663),
  'Value': 0,
  'Geometry': <POINT (4.637 51.789)>},
 {'Message': "Sailing from node ('NL', 'J2676') to node (

### Visualization of path
If you get an error regarding ffmpeg use [this](https://stackoverflow.com/questions/13316397/matplotlib-animation-no-moviewriters-available) answer. You have to install ffmpeg in your Conda environment. It can be done using the following command.

```bash
#Install ffmpeg using Conda
conda install -c conda-forge ffmpeg
```

In [ ]:
# visualise vessel movements based on the information included in the vessel.log
# plot.vessel_kml(env, vessels, stepsize = 60)
#plot.graph_kml(env)


In [ ]:
! start explorer .